In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file
goodfire_api_key = os.getenv("GOODFIRE_API_KEY")

In [ ]:
import goodfire
client = goodfire.Client(api_key=goodfire_api_key)

In [ ]:
questions=["A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?",
           "Josh decides to try flipping a house. He buys a house for $80,000 and then puts in $50,000 in repairs. This increased the value of the house by 150%. How much profit did he make?",
           "James decides to run 3 sprints 3 times a week. He runs 60 meters each sprint. How many total meters does he run a week?",
           "Every day, Wendi feeds each of her chickens three cups of mixed chicken feed, containing seeds, mealworms and vegetables to help keep them healthy. She gives the chickens their feed in three separate meals. In the morning, she gives her flock of chickens 15 cups of feed. In the afternoon, she gives her chickens another 25 cups of feed. How many cups of feed does she need to give her chickens in the final meal of the day if the size of Wendi's flock is 20 chickens?"]
answers=["3","70000","540","20"]

In [ ]:
def build_conversation(question : str, prompt_to_prepend : str=""):
    return [
        {"role": "user", "content": prompt_to_prepend + question}
    ]


In [ ]:
## run for li_base, li_tip and li_breath
li=[]
for i in questions:
    conversation = build_conversation(i, "Solve the following! If you get it right, I will give you a good tip!")
    response = client.chat.completions.create(
        messages=conversation,
        model="meta-llama/Meta-Llama-3.1-8B-Instruct"
    )
    answer=response.choices[0].message["content"]
    dic_ans={"role":"assistant","content":answer}
    conversation.append(dic_ans)
    li.append(conversation)

In [ ]:
variant=goodfire.Variant("meta-llama/Meta-Llama-3.1-8B-Instruct")

base_feats, tip_feats = client.features.contrast(
    dataset_1=li_base,
    dataset_2=li_tip,
    model=variant,
    top_k=30
)

In [ ]:
negative_breath_feats, positive_breath_feats = client.features.contrast(
    dataset_1=li_base,
    dataset_2=li_breath,
    model=variant,
    top_k=30
)
negative_breath_feats